# Predicción del Costo de Taxi en NYC usando solo Pandas y Numpy

Este notebook hace un modelo muy simple para predecir el costo de un viaje en taxi usando sólo `pandas` y `numpy`, como práctica didáctica.

In [ ]:

# === 1. IMPORTACIÓN DE LIBRERÍAS ===
import pandas as pd
import numpy as np


In [ ]:

# === 2. CARGA DEL DATASET ===
df = pd.read_csv("train.csv", nrows=50000)  # Leemos solo 50.000 para mayor rapidez
df = df.dropna()  # Eliminamos filas con datos faltantes
df.head()


In [ ]:

# === 3. FILTRADO DE VALORES EXTRAÑOS ===
# Solo tarifas positivas y razonables
df = df[(df['fare_amount'] > 0) & (df['fare_amount'] < 200)]

# Coordenadas dentro de NYC aproximado
df = df[
    (df['pickup_latitude'].between(40, 42)) &
    (df['pickup_longitude'].between(-75, -72)) &
    (df['dropoff_latitude'].between(40, 42)) &
    (df['dropoff_longitude'].between(-75, -72))
]


In [ ]:

# === 4. CREACIÓN DE VARIABLES ===
# Distancia euclidiana como aproximación
df['distancia'] = np.sqrt(
    (df['pickup_latitude'] - df['dropoff_latitude'])**2 +
    (df['pickup_longitude'] - df['dropoff_longitude'])**2
)


In [ ]:

# === 5. MODELO SIMPLE CON NÚMEROS BASE ===

# Usamos regresión lineal manual: y = b0 + b1 * distancia + b2 * pasajeros
# Para encontrar los coeficientes, usamos una solución con matrices (mínimos cuadrados)

# Matriz X con columna de 1s (intercepto), distancia y pasajeros
X = np.column_stack((np.ones(len(df)), df['distancia'], df['passenger_count']))
y = df['fare_amount'].values

# Fórmula de mínimos cuadrados: beta = (X^T X)^(-1) X^T y
beta = np.linalg.inv(X.T @ X) @ X.T @ y
beta


In [ ]:

# === 6. PRUEBA DE PREDICCIÓN ===
# Tomamos 5 ejemplos y predecimos

ejemplo = X[:5]
predicciones = ejemplo @ beta
print("Predicciones:", predicciones)
print("Valores reales:", y[:5])


In [ ]:

# === 7. ERROR CUADRÁTICO MEDIO (RMSE) ===
pred_todas = X @ beta
rmse = np.sqrt(np.mean((pred_todas - y)**2))
print(f"RMSE del modelo lineal simple: {rmse:.2f}")
